<a href="https://colab.research.google.com/github/oscarfdezmora/Coursera_Capstone/blob/master/Capstone_Project_The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The problem**

A firm of medical supplies wants to open a new pharmacy in Toronto, and is interested in looking for new opportunities Scarborough.  

Taking into account the density of this kind of commerces in the area, we will offer a set most likely boroughs to open their store. 

This kind of study would be suitable for profiling any kind of store or venue in a sets of address, by means of geospatial tools such as Foursquare or Google Maps.

# **The Data**

We will focus on thos nighborhoods which are address in **North York, Toronto**. This locations will be provided by the postal code information settled in Wikipedia (source: [List of postal codes of Canada: M](https://www.wikiwand.com/en/List_of_postal_codes_of_Canada:_M))

The **Foursquare API** will be use to find locations and look for those area which lack of pharmacies in the nearbies



# **Data Preparation**


Python libraries and dependencies

In [ ]:
!pip install geocoder
!pip install folium

In [30]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Data Extraction and Cleaning


In [ ]:
## Get Data from Wikipedia
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

In [ ]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.head()

In [ ]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
df.head()

In [34]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']